In [ ]:
#헤드폰 연결 및 IMU, UWB 데이터 수치화
import time
import serial
import serial.tools.list_ports 

for port in serial.tools.list_ports.comports():
    print(port.device)

ser = serial.Serial("/dev/cu.usbserial-110", 57600)
print(ser.name)

In [ ]:
def read_until_full(ser=ser, timeout=None):
    ser.timeout = timeout
    ser.flush()
    ser.flushInput()
    while True:
        readed = ser.read(size=1)
        if readed==b'\xff':
            incomming = ser.read(size=6)
            break 
            
    return incomming

In [ ]:
hex_incomming = read_until_full()

[x for x in hex_incomming]

def get_values(hex_incomming):
    """    x = hex_incomming[0] / 10
    y = hex_incomming[1] / 10""" 
    x = hex_incomming[1] / 10
    y = hex_incomming[0] / 10 
    heading = (hex_incomming[2]*255 + hex_incomming[3]) / 100
    return x, y, heading

get_values(hex_incomming)

def get_value_onestep():
    hex_incomming = [x for x in read_until_full()]
    #print('hex_incomming :', hex_incomming)
    return get_values(hex_incomming)

get_value_onestep()


In [ ]:
#라이브러리 선언
from flask import Flask, render_template, request, jsonify
import os
import openai
from google.cloud import speech
from google.cloud import texttospeech
import numpy as np
import pyaudio
import wave
import pygame
import math
import time

In [ ]:
#GPT-4 사용을 위한 인증키 가져오기
openai.api_key = "Your API Key"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Your JSON Path"

In [ ]:
#Flask app 생성
app = Flask(__name__)
count = 0

In [ ]:
#####STT, TTS 음성 대화를 위한 코드

#사용자 음성 녹음
def record_audio():
    CHUNK = 1024
    FORMAT = pyaudio.paInt16  # 16 bit
    CHANNELS = 1  # mono
    RATE = 16000  # sample rate
    SILENCE_THRESHOLD = 300  # silence threshold
    SILENT_CHUNKS = int(2.5 * RATE / CHUNK)  # 10초 동안 소리 없으면 out
    MAX_RECORD_SECONDS = 5  # maximum recording time in seconds
    
    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* recording")

    frames = []
    silent_chunks = 0
    audio_started = False
    start_time = None

    while True:
        data = stream.read(CHUNK)
        frames.append(data)

        # Check for silence
        audio_data = np.frombuffer(data, dtype=np.int16)
        if np.abs(audio_data).mean() < SILENCE_THRESHOLD:
            silent_chunks += 1
            if silent_chunks > SILENT_CHUNKS:
                break
        else:
            print(np.abs(audio_data).mean())
            silent_chunks = 0   

        if audio_started:
            if silent_chunks > SILENT_CHUNKS or (time.time() - start_time) >= MAX_RECORD_SECONDS:
                break
        elif silent_chunks == 0:
            audio_started = True
            start_time = time.time()

    print("* done recording")

    stream.stop_stream()
    stream.close()
    p.terminate()

    # Save the recorded audio to a WAV file
    wf = wave.open("audio.wav", "wb")
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b"".join(frames))
    wf.close()

    # Return the recorded audio data
    return b"".join(frames)

## Byte 오디오 데이터를 한글 text로 변환 STT
def transcribe_audio(audio_data):
    audio = speech.RecognitionAudio(content=audio_data)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="ko-KR"
    )

    response = stt_client.recognize(config=config, audio=audio)
    
    if response.results:
        return response.results[0].alternatives[0].transcript
    else:
        print("음성 인식에 실패했습니다. 다시 시도해주세요.")
        return None
    
## 텍스트를 음성으로 변환
def synthesize_speech(text, voice_index=0):
    # Ensure the provided index is valid
    if voice_index < 0 or voice_index >= len(speech_params_list):
        raise ValueError("Invalid voice index provided")

    # Choose the voice settings based on the provided index
    speech_params = speech_params_list[voice_index]

    # Convert the ssml_gender string to the corresponding enum value
    ssml_gender_map = {
        "MALE": texttospeech.SsmlVoiceGender.MALE,
        "FEMALE": texttospeech.SsmlVoiceGender.FEMALE,
    }
    ssml_gender = ssml_gender_map[speech_params["ssml_gender"]]

    input_text = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="ko-KR",
        ssml_gender=ssml_gender,  # Use the enum value instead of string
        name=speech_params["voice_name"]
    )
    
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.LINEAR16,
        speaking_rate=speech_params["speaking_rate"],
        pitch=speech_params["pitch"]
    )

    response = tts_client.synthesize_speech(
        input=input_text, voice=voice, audio_config=audio_config
    )

    return response.audio_content

## pyaudio를 사용하여 음성을 재생하는 함수
def play_audio(audio_data):
    p = pyaudio.PyAudio()
    stream = p.open(
        format=p.get_format_from_width(2),
        channels=1,
        rate=24000,
        output=True,
    )
    stream.write(audio_data)
    stream.stop_stream()
    stream.close()
    p.terminate()

# Set up STT and TTS API clients 변환된 데이터값 변수로 받기
stt_client = speech.SpeechClient()
tts_client = texttospeech.TextToSpeechClient()

# 프롬프트설계, 과거기억 데이터 확인
messages = []

In [ ]:
## 목소리 설정하기
# 남자 목소리 name 변수 수정 = ko-KR-Neural2-C, ko-KR-Standard-C(D), ko-KR-Wavenet-C(D)
# 여자 목소리 name 변수 수정 = ko-KR-Neural2-A(B), ko-KR-Standard-A(B), ko-KR-Wavenet-A(B)
# language_code="ko-KR",ssml_gender=texttospeech.SsmlVoiceGender.FEMALE, name="ko-KR-Standard-A"
# language_code="ko-KR",ssml_gender=texttospeech.SsmlVoiceGender.MALE, name="ko-KR-Neural2-C"

speech_params_list = [
    {   #0 
        "speaking_rate": 1.1,
        "pitch": -2.0,
        "ssml_gender": "MALE",
        "voice_name": "ko-KR-Standard-C"
    },
    {   #1 
        "speaking_rate": 1.0,
        "pitch": 0.0,
        "ssml_gender": "FEMALE",
        "voice_name": "ko-KR-Standard-A"
    } 
]

In [ ]:
#while문 사용을 위한 헤딩값 재계산
def get_heading():
    return get_value_onestep()[2]

#목표 스팟에 대한 거리값 계산 - 방향과 거리에 대한 큐를 주기 위한 값
def get_distance1(x, y):
    x1, y1 = 8.0, 3.3  # 어른
    return math.sqrt((x1 - x) ** 2 + (y1 - y) ** 2)
def get_distance2(x, y):
    x2, y2 = 5.8, 0.7  # 마음
    return math.sqrt((x2 - x) ** 2 + (y2 - y) ** 2)
def get_distance3(x, y):
    x3, y3 = 4.2, 4.2  # 재구성
    return math.sqrt((x3 - x) ** 2 + (y3 - y) ** 2)
def get_distance4(x, y):
    x4, y4 = 2.0, 2.2  # 해체
    return math.sqrt((x4 - x) ** 2 + (y4 - y) ** 2)

In [ ]:
# Zone 정보 및 좌표 설정
zones_f1 = {
    #구역
    "Zone1": [(9.4, 0.0), (11.0, 6.5)],  # 시간
    "Zone1-1": [(7.2, 4.3), (9.4, 6.5)],   # 시간
    "Zone2": [(7.2, 0.0), (9.4, 4.3)],   # 어른
    "Zone3": [(3.5, 0.0), (7.0, 4.0)],   # 마음
    "Zone4": [(0.0, 3.5), (4.6, 6.5)],   # 재구성 4.6, 4.6
    "Zone5": [(0.0, 0.0), (3.5, 3.5)]   # 해체
} 
# 사운드 파일 재생 여부를 추적하기 위한 딕셔너리
sound_played = {
    "rec2_1": False,
    "rec2_2": False,
    "rec2_3": False,
    "rec3_1": False,
    "rec3_2": False,
    "rec3_3": False,
    "rec4_1": False,
    "rec4_2": False,
    "rec5_1": False,
    "rec5_2": False
}
conv_executed = {
    "conv1": False,
    "conv2": False,
    "conv3": False,
    "conv4": False
}

In [ ]:
def play_bgm(path, volume=0.5):
    pygame.mixer.init()
    pygame.mixer.music.load(path)
    pygame.mixer.music.set_volume(volume)  # 볼륨 설정
    pygame.mixer.music.play(0)  # -1은 무한 반복, 0은 한번 재생을 의미

def play_conv(path, callback=None):
    pygame.mixer.init()
    pygame.mixer.music.load(path)
    pygame.mixer.music.play(0)  # 0은 파일을 한 번만 재생합니다.

    while pygame.mixer.music.get_busy():
        time.sleep(0.1)  # 음악 재생 상태 확인을 위한 잠시 대기

    if callback:  # 재생이 완료된 후 실행할 함수가 있다면 호출
        callback()

def stop_bgm():
    pygame.mixer.music.stop()

In [ ]:
def get_zone(x, y):
    for zone, (start, end) in zones_f1.items():
        if start[0] <= x <= end[0] and start[1] <= y <= end[1]:  # 조건 수정
            return zone
    return None

zone_N = get_zone(get_value_onestep()[0], get_value_onestep()[1])
print(get_value_onestep()[0], get_value_onestep()[1])
print(zone_N)

In [ ]:
# 시간 
rec1_bgm = "File Path"
# 어른 녹음본 2개 각도
rec2_1 = "File Path"
rec2_2 = "File Path"
rec2_3 = "File Path"
rec2_bgm = "File Path"
# 마음 녹음본 2개 각도
rec3_1 = "File Path"
rec3_2 = "File Path"
rec3_3 = "File Path"
rec3_bgm = "File Path"
# 재구성 녹음본 2개 각도
rec4_1 = "File Path"
rec4_2 = "File Path"
rec4_bgm = "File Path"
# 해제 녹음본 2개 각도
rec5_1 = "File Path"
rec5_2 = "File Path"
rec5_bgm = "File Path"


In [ ]:
import requests
import io
from pydub import AudioSegment
from pydub.playback import play

In [ ]:
#elevenlabs
def synthesize(text):
    url = "https://api.elevenlabs.io/v1/text-to-speech/"
    headers = {
        "Accept": "audio/mpeg",
        "Content-Type": "application/json",
        "xi-api-key": "You API Key"  # 여기에 실제 API 키를 입력해야 합니다. 내 키
    }
    data = {
        "text": text,
        "model_id": "eleven_multilingual_v2",
        "voice_settings": {
            "stability": 0.5,
            "similarity_boost": 0.75,
            "style_exaggeration": 0.0
        }
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        audio_stream = io.BytesIO(response.content)
        audio = AudioSegment.from_file(audio_stream, format="mp3")
        play(audio)
    else:
        print("Error: ", response.status_code)

In [ ]:
#GPT 사용 정의
assistant_role = "닥터로만 실험실 가이드, 아이"
assistant_persona = "무섭고 궁금증을 자아내는"
user_role = "방문객"
messages.append({"role": "system", "content": f"GPT(:{assistant_role})와 사용자(:{user_role})의 역할극을 시작합니다. {assistant_role}은 {assistant_persona} 성격으로 '닥터로만 실험실' 관람에 대해 스몰토크를 합니다."})
assistant_content = None

In [ ]:
#음향 재생 후 대화
def conv1():  
    if conv_executed["conv1"]:
        return  # 이미 실행된 경우 함수를 종료

    print("어른 대화 1 ")
    conv_1 = "안녕하세요. 아이 입니다. 박사님은 집요하게, 인간의 의식에 대해 집착을 했어요. 당신은 당신의 의식을 완전히 통제할 수 있다고 믿나요?"
    synthesize(conv_1) #elevenlabs API TTS
    messages = [{"role": "system", "content": conv_1}] #conv_GPT 추가

    print("어른 대화 1-1 ")
    user_audio = record_audio()
    user_content = transcribe_audio(user_audio)
    print(user_content)

    if not user_content:
        user_content = "모르겠어."
    messages.append({"role": "system", "content": "다음 아래 사용자 발화에 대해 총 두 문장으로 대답해줘. 마지막 문장은 질문으로 대답해줘"})
    messages.append({"role": "user", "content": user_content}) #사용자 발화 추가

    # 대답1 Use GPT API to generate response
    completion = openai.ChatCompletion.create(model="gpt-4", messages=messages)
    assistant_content = completion.choices[0].message["content"].strip()
    messages.append({"role": "assistant", "content": f"{assistant_content}"})
    print(f"S.I : {assistant_content}")

    print("어른 대화 2 ")
    synthesize(assistant_content) #elevenlabs API TTS

    print("어른 대화 2-1 ")
    user_audio = record_audio()
    user_content = transcribe_audio(user_audio)
    print(user_content)

    print("어른 대화 3 ")
    ## 대답2 Use GPT API to generate response
    completion = openai.ChatCompletion.create(model="gpt-4", messages=messages)
    assistant_content = completion.choices[0].message["content"].strip()
    messages.append({"role": "assistant", "content": f"{assistant_content}"})
    print(f"S.I : {assistant_content}")
    synthesize(assistant_content) #elevenlabs API TTS

    user_audio = record_audio()
    user_content = transcribe_audio(user_audio)
    return None 

def conv2():  
    if conv_executed["conv2"]:
        return  # 이미 실행된 경우 함수를 종료

    print("마음 대화 1 ")
    conv_1 = "이 조각상은 마음이라는 작품이에요. 음.. 박사님은 인간의 마음조차 재현할 수 있다고 믿으셨죠. 당신은 마음이란 것이 존재한다고 믿으시나요?"
    synthesize(conv_1) #elevenlabs API TTS
    messages = [{"role": "system", "content": conv_1}] #conv_GPT 추가

    print("마음 대화 1-1 ")
    user_audio = record_audio()
    user_content = transcribe_audio(user_audio)
    print(user_content)

    if not user_content:
        user_content = "모르겠어."
    messages.append({"role": "system", "content": "다음 아래 사용자 발화에 대해 총 두 문장으로 대답해줘. 마지막 문장은 질문으로 대답해줘"})
    messages.append({"role": "user", "content": user_content}) #사용자 발화 추가

    # 대답1 Use GPT API to generate response
    completion = openai.ChatCompletion.create(model="gpt-4", messages=messages)
    assistant_content = completion.choices[0].message["content"].strip()
    messages.append({"role": "assistant", "content": f"{assistant_content}"})
    print(f"S.I : {assistant_content}")

    print("마음 대화 2 ")
    synthesize(assistant_content) #elevenlabs API TTS

    print("마음 대화 2-1 ")
    user_audio = record_audio()
    user_content = transcribe_audio(user_audio)
    print(user_content)

    print("마음 대화 3 ")
    ## 대답2 Use GPT API to generate response
    completion = openai.ChatCompletion.create(model="gpt-4", messages=messages)
    assistant_content = completion.choices[0].message["content"].strip()
    messages.append({"role": "assistant", "content": f"{assistant_content}"})
    print(f"S.I : {assistant_content}")
    synthesize(assistant_content) #elevenlabs API TTS

    user_audio = record_audio()
    user_content = transcribe_audio(user_audio)
    return None 

def conv3():  
    if conv_executed["conv3"]:
        return  # 이미 실행된 경우 함수를 종료

    print("재구성 대화 1 ")
    conv_1 = "이 작품의 이름은 재구성 입니다. 뉴런 신호를 데이터화 한다는 박사님의 영혼이 담겨있죠. 당신은 이 세상이 실재한다고 믿나요?, 결국 이 세상도 하나의 가상 현실이 아닐까요?"
    synthesize(conv_1) #elevenlabs API TTS
    messages = [{"role": "system", "content": conv_1}] #conv_GPT 추가

    print("재구성 대화 1-1 ")
    user_audio = record_audio()
    user_content = transcribe_audio(user_audio)
    print(user_content)

    if not user_content:
        user_content = "모르겠어."
    messages.append({"role": "system", "content": "다음 아래 사용자 발화에 대해 총 두 문장으로 대답해줘. 마지막 문장은 질문으로 대답해줘"})
    messages.append({"role": "user", "content": user_content}) #사용자 발화 추가

    # 대답1 Use GPT API to generate response
    completion = openai.ChatCompletion.create(model="gpt-4", messages=messages)
    assistant_content = completion.choices[0].message["content"].strip()
    messages.append({"role": "assistant", "content": f"{assistant_content}"})
    print(f"S.I : {assistant_content}")

    print("재구성 대화 2 ")
    synthesize(assistant_content) #elevenlabs API TTS

    print("재구성 대화 2-1 ")
    user_audio = record_audio()
    user_content = transcribe_audio(user_audio)
    print(user_content)

    print("재구성 대화 3 ")
    ## 대답2 Use GPT API to generate response
    completion = openai.ChatCompletion.create(model="gpt-4", messages=messages)
    assistant_content = completion.choices[0].message["content"].strip()
    messages.append({"role": "assistant", "content": f"{assistant_content}"})
    print(f"S.I : {assistant_content}")
    synthesize(assistant_content) #elevenlabs API TTS

    user_audio = record_audio()
    user_content = transcribe_audio(user_audio)
    return None 

def conv4():  
    if conv_executed["conv4"]:
        return  # 이미 실행된 경우 함수를 종료

    print("해체 대화 1 ")
    conv_1 = "몸과 팔, 다리. 이 작품은 기계와 인간을 같은 레이어로 바라보는 박사님의 사상이 담겨있습니다. 당신도 스스로를 기계같다고 느낀 적이 있나요?"
    synthesize(conv_1) #elevenlabs API TTS
    messages = [{"role": "system", "content": conv_1}] #conv_GPT 추가

    print("해체 대화 1-1 ")
    user_audio = record_audio()
    user_content = transcribe_audio(user_audio)
    print(user_content)

    if not user_content:
        user_content = "모르겠어."
    messages.append({"role": "system", "content": "다음 아래 사용자 발화에 대해 총 두 문장으로 대답해줘. 마지막 문장은 질문으로 대답해줘"})
    messages.append({"role": "user", "content": user_content}) #사용자 발화 추가

    # 대답1 Use GPT API to generate response
    completion = openai.ChatCompletion.create(model="gpt-4", messages=messages)
    assistant_content = completion.choices[0].message["content"].strip()
    messages.append({"role": "assistant", "content": f"{assistant_content}"})
    print(f"S.I : {assistant_content}")

    print("해체 대화 2 ")
    synthesize(assistant_content) #elevenlabs API TTS

    print("해체 대화 2-1 ")
    user_audio = record_audio()
    user_content = transcribe_audio(user_audio)
    print(user_content)

    print("해체 대화 3 ")
    ## 대답2 Use GPT API to generate response
    completion = openai.ChatCompletion.create(model="gpt-4", messages=messages)
    assistant_content = completion.choices[0].message["content"].strip()
    messages.append({"role": "assistant", "content": f"{assistant_content}"})
    print(f"S.I : {assistant_content}")
    synthesize(assistant_content) #elevenlabs API TTS

    user_audio = record_audio()
    user_content = transcribe_audio(user_audio)
    return None 

conv_end1_counter = 0

def conv_end():
    global conv_end1_counter
    conv_end1_counter += 1

    if conv_end1_counter == 1:
        print("작품1 관람 완료")
        conv_continue = "이 작품은 여기까지하고 다른 작품도 둘러보기로 하죠. 또 다른 박사님의 생각을 읽을 수 있을 겁니다."
        synthesize(conv_continue)  # elevenlabs API TTS
    elif conv_end1_counter == 2:
        print("작품2 관람 완료")
        conv_continue = "음.. 저는 잘 모르겠습니다. 다른 작품도 천천히 살펴 보시죠."
        synthesize(conv_continue)  # elevenlabs API TTS
    elif conv_end1_counter == 3:
        print("작품3 관람 완료")
        conv_continue = "무슨 생각을 하시든, 저에게는 새롭네요. 남은 작품도 같이 볼까요."
        synthesize(conv_continue)  # elevenlabs API TTS
    elif conv_end1_counter == 4:
        conv_end = "준비된 작품들은 여기까지입니다. 저희 새로운 조수들에게 헤드셋을 반납해 주세요."
        synthesize(conv_end)  # elevenlabs API TTS
    return None

In [ ]:
conv_g = "안녕하세요. 반갑습니다. 경험 시작 전, 내레이션이 들리면 손가락으로 동그라미를 표시해 주세요!"
start_guide = synthesize_speech(conv_g, voice_index=1)
play_audio(start_guide)

In [ ]:
current_bgm = None  # 현재 재생 중인 배경 음악을 추적하기 위한 변수
current_zone = None  # 이전 zone을 추적하기 위한 변수
previous_zone = None

conv_s = "안녕하세요. 인공지능 시스템 아이입니다. 이곳에는 박사님이 만든 여러 조각 작품이 있어요. 작품 근처에서는 박사님의 생각을 들을 수 있습니다...,박사님의 생각을 듣다가 중간중간 저와 대화를 나누어 보아요. 작품 주변을 천천히 한바퀴 둘러보시며 소리에 귀기울여 보시기 바래요. 즐거운 관람 되시기 바랍니다."
synthesize(conv_s)

while True:
    array_t = get_value_onestep()
    pre_x = array_t[0]
    pre_y = array_t[1]
    heading = array_t[2]

    current_zone = get_zone(pre_x, pre_y)
    """dis1 = get_distance1(pre_x, pre_y)
    dis2 = get_distance2(pre_x, pre_y)
    dis3 = get_distance3(pre_x, pre_y)
    dis4 = get_distance4(pre_x, pre_y)"""

    print("헤딩:", heading) #연구실 환경에 맞게 0도를 화이트보드 방향으로 
    print("좌표:", pre_x, ",", pre_y)
    #print("스팟과의 거리:",dis1, ",", dis2, ",", dis3, ",", dis4)
    print(f"현재위치: {current_zone}.")

    # 현재 Zone 결정 및 이전 Zone 추적
    if current_zone != previous_zone:
        previous_zone = current_zone  # 현재 Zone을 이전 Zone으로 업데이트

        if current_zone == "Zone1":
            play_bgm(rec1_bgm)
            
        elif current_zone == "Zone2":  #어른
            print("check1")
            play_bgm(rec2_bgm)
            print("check2")
            time.sleep(5.0)
            if not sound_played["rec2_1"]:
                sound_played["rec2_1"] = True
                play_conv(rec2_1)
            if 0 <= heading <= 180 and not sound_played["rec2_2"]:
                sound_played["rec2_2"] = True
                stop_bgm()
                play_conv(rec2_2)
                time.sleep(2.0)
                play_conv(rec2_3)
                play_bgm(rec2_bgm)
                conv1()
                conv_end()
            else:
                play_bgm(rec2_bgm)

        elif current_zone == "Zone3":  #마음
            play_bgm(rec3_bgm)
            print("check3")
            time.sleep(5.0)
            if not sound_played["rec3_1"]:
                sound_played["rec3_1"] = True
                play_conv(rec3_1)
            if 0 <= heading <= 180 and not sound_played["rec3_2"]:
                sound_played["rec3_2"] = True
                stop_bgm()
                play_conv(rec3_2)
                time.sleep(2.0)
                play_conv(rec3_3)
                play_bgm(rec3_bgm)
                conv2()
                conv_end()
            else:
                play_bgm(rec3_bgm)
                
        elif current_zone == "Zone4":  #재구성
            play_bgm(rec4_bgm)
            print("check4")
            time.sleep(5.0)
            if not sound_played["rec4_1"]:
                sound_played["rec4_1"] = True
                play_conv(rec4_1)
            if 0 <= heading <= 180 and not sound_played["rec4_2"]:
                sound_played["rec4_2"] = True
                stop_bgm()
                play_conv(rec4_2)
                play_bgm(rec4_bgm)
                conv3()
                conv_end()
            else:
                play_bgm(rec4_bgm)
                    
        elif current_zone == "Zone5":  #해체
            play_bgm(rec5_bgm)
            print("check5")
            time.sleep(5.0)
            play_conv(rec5_1)
            if not sound_played["rec5_1"]:
                sound_played["rec5_1"] = True
                play_conv(rec5_1)
            if 0 <= heading <= 180 and not sound_played["rec5_2"]:
                sound_played["rec5_2"] = True
                stop_bgm()
                play_conv(rec5_2)
                play_bgm(rec5_bgm)
                conv4()
                conv_end()
            else:
                play_bgm(rec5_bgm)

In [ ]:
stop_bgm()
#stop_sound_effect()

In [ ]:
current_bgm = None  # 현재 재생 중인 배경 음악을 추적하기 위한 변수
current_zone = None  # 이전 zone을 추적하기 위한 변수
previous_zone = None
while True:
    array_t = get_value_onestep()
    pre_x = array_t[0]
    pre_y = array_t[1]
    heading = array_t[2]

    current_zone = get_zone(pre_x, pre_y)
    """dis1 = get_distance1(pre_x, pre_y)
    dis2 = get_distance2(pre_x, pre_y)
    dis3 = get_distance3(pre_x, pre_y)
    dis4 = get_distance4(pre_x, pre_y)"""

    print("헤딩:", heading) #연구실 환경에 맞게 0도를 화이트보드 방향으로 
    print("좌표:", pre_x, ",", pre_y)
    #print("스팟과의 거리:",dis1, ",", dis2, ",", dis3, ",", dis4)
    print(f"현재위치: {current_zone}.")

    # 현재 Zone 결정 및 이전 Zone 추적
    if current_zone != previous_zone:
        previous_zone = current_zone  # 현재 Zone을 이전 Zone으로 업데이트

        if current_zone == "Zone1":
            play_bgm(rec1_bgm)
            
        elif current_zone == "Zone2":  #어른
            print("check2")
            play_bgm(rec2_bgm)
        elif current_zone == "Zone3":  #어른
            print("check3")
            play_bgm(rec3_bgm)
        elif current_zone == "Zone4":  #어른
            print("check4")
            play_bgm(rec4_bgm)
        elif current_zone == "Zone5":  #어른
            print("check5")
            play_bgm(rec5_bgm)